# Integrating the Friedmann Equation


Objective: To get a feel for how different choices of $\Omega_{m,0}$, $\Omega_{\Lambda,0}$, $\Omega_{r,0}$ affect the evolution and fate of a universe.  
1. Let's recreate plot 6.3 from Ryden. This shows $a(t)$ from $t = 1e-10 H_0^{-1}$ to the present day. Do we get the correct scaling behaviors in the radiation, matter, and $\Lambda$-dominated regimes?
2. When do we switch from each regime to the next? How long ago did the shifts occur? At what redshift?
3. What happens as we look forward in time? What choices of $\Omega_{m,0}$, $\Omega_{\Lambda,0}$, and $\Omega_{r,0}$ lead to big crunches or endless expansion?
4. How would the evolution change if the equations of state were different?

The code below can only handle expanding universes ($\dot{a} \ge 0$). What changes would you have to make to allow for compression?

There's no definite endpoint to this activity, but instead to build some intuition of the consequences of different compositions.

First, let's load a couple useful packages

In [1]:
import numpy as np
from scipy.interpolate import interp1d

Step 1: Compute $\dot{a}(a,\Omega_i)$

In [2]:
def aDot(a,Omegas,Ws):
    # Compute the sum over components
    Val = 0.
    for i in range(len(Omegas)):
        Val += Omegas[i] * a**(-3 * (1 + Ws[i]))

    ## Add the curvature term
    
    Omega0 = 0.
    for Omega in Omegas:
        Omega0 += Omega

    Val += (1 - Omega0) / a**2
    
    ## Take a square root:
    Val = (Val)**0.5
    
    ## Multiply by a
    Val *= a
    
    return(Val)

Let's do a simple numerical integral. We know the value at a particular time ($a(t_0) = 1$). A simple first-order integration should be sufficient for our task. We'll have to do a pair of integrations since our known point is in the middle of our integration range. We should use a variable timestep since the timescale ($H^{-1}$) is changing with time. For this timestep, let's choose $0.01 H^{-1}$. 

In [3]:
def integrateFriedmann(Omegas,Ws,nT=int(1e4)):

    # Start lists of the various A's and T's that we'll find.
    
    ForwardAs = [1.]
    ForwardTs = [0.]
    BackwardAs = [1.]
    BackwardTs = [0.]
    
    # Integrate forward
    for i in range(nT):
        # Find the derivative at the current a
        currentaDot = aDot(ForwardAs[-1],Omegas,Ws)
        
        # Set the timestep
        foredT = 0.01 * (ForwardAs[-1] / currentaDot)
        
        # Find the next a and t with a very simple first-order linear scheme
        ForwardAs.append(ForwardAs[-1] + currentaDot * foredT)
        ForwardTs.append(ForwardTs[-1] + foredT)

    # Integrate backward
    for i in range(nT):
        # Find the derivative at the current a
        currentaDot = aDot(BackwardAs[-1],Omegas,Ws)
        
        # Set the timestep
        backdT = 0.01 * (BackwardAs[-1] / currentaDot)
        
        # Find the next a and t with a very simple first-order linear scheme
        BackwardTs.append(BackwardTs[-1] - backdT)
        BackwardAs.append(BackwardAs[-1] - currentaDot * backdT)

    # Collect everything
    Ts = []
    As = []

    for i in range(1,len(BackwardTs)+1):
        Ts.append(BackwardTs[-i])
        As.append(BackwardAs[-i])
    for i in range(len(ForwardTs)):
        Ts.append(ForwardTs[i])
        As.append(ForwardAs[i])
        
    return(Ts,As)


Let's also compute the three equality times, where the energy density due to pairs of components are equal.

In [4]:
def equalityAs(Omegas, Ws,Names):
    # Create a list of the scale factors when pairs of components have equal density
    
    labels = []
    As = []
    
    # Loop through the components
    for i in range(len(Omegas)):
        # Loop through the other components
        for j in range(i+1,len(Omegas)):
            
            # Give the time a name
            labels.append(Names[i] + ', ' + Names[j])
            
            # Compute the equality time
            As.append(np.exp((np.log(Omegas[i] / Omegas[j])) / (-3 * (Ws[j] - Ws[i]))))
            
    
    return(As,labels)

Let's load a couple packages for plotting. 

In [5]:
%matplotlib notebook
from ipywidgets import interact, interactive,  fixed
from IPython.display import clear_output, display, HTML

import matplotlib.pyplot as plt

Finally, let's make a plot. This will be interactive with several variables:
* fracL is a scale factor setting the amount of dark energy ($\Lambda$, $w = -1$)
* fracM is a scale factor setting the amount of matter ($w = 0$)
* logfracR is a scale factor setting the amount of radiation ($w = 1/3$)

These three variables set the relative amounts of the three components.
* Omega0 sets the total amount of mass in the usual way ($\Omega_0 = 1$ is a flat universe)
The last two variables don't affect the integration at all, but just shift the two guide-lines up or down.
* radScale shifts the $a \propto t^{1/2}$ line up or down
* matterScale shifts the $a \propto t^{2/3}$ line up or down

In [6]:
@interact(fracL=0.70, fracM=0.30, logfracR=-4,Omega0 = 1.0,radScale=0.15, matterScale = 0.80)
def plotAT(fracL=0.70, fracM=0.30, logfracR=-4,Omega0 = 1.0,radScale=0.15, matterScale = 0.80):

    # Find the scale factor for the densities
    OmegaScale = Omega0 / (fracL + fracM + 10**logfracR)
    
    Omegas = []
    Ws = []
    Names = []
    
    # Only add dark energy and matter if they have nonzero fractions
    if fracL != 0.:
        Omegas.append(fracL * OmegaScale)
        Ws.append(-1.)
        Names.append("L")
    if fracM != 0.:
        Omegas.append(fracM * OmegaScale)
        Ws.append(0.)
        Names.append("M")
        
    Omegas.append(OmegaScale * 10**logfracR)
    Ws.append(1./3.)
    Names.append("R")
    
    # Integrate the Friedmann equation for the choice of Omegas
    Ts, As = integrateFriedmann(Omegas, Ws)
    
    # Construct an interpolating function for a(t)
    aTfn = interp1d(As,Ts - np.min(Ts))
    
    # Create a list of the times when components have equal densities
    EAs, Labels = equalityAs(Omegas, Ws, Names)


    # Build the plot
    fig, axes = plt.subplots()
    
    # Plot a(t). Use the first T value as t = 0
    axes.plot(np.subtract(Ts,np.min(Ts)),As, label="$a(t)$")
    
    # Plot the matter and radiation scaling lines
    axes.plot(np.subtract(Ts,np.min(Ts)), radScale * np.subtract(Ts,np.min(Ts))**0.5, label="$\propto t^{1/2}$",alpha=0.5)
    axes.plot(np.subtract(Ts,np.min(Ts)), matterScale * np.subtract(Ts,np.min(Ts))**(2./3.), label="$\propto t^{2/3}$",alpha=0.5)

    # Plot horizontal and vertical lines at the a and t when components have equal densities
    for i in range(len(EAs)):
        axes.axhline(EAs[i],linestyle=':')
        axes.axvline(aTfn(EAs[i]),linestyle=':')
        
    # Plot a horizontal and vertical line passing through a = 1, t = t0
    axes.axhline(1.0,linestyle=':')
    axes.axvline(1.0,linestyle=':')

    # Set the plot scales
    axes.set_yscale('log')
    axes.set_xscale('log')
    axes.set_ylim(1e-7,1e2)
    axes.set_xlim(1e-10,1e1)
    axes.set_xlabel("H_0 t")
    axes.set_ylabel("a(t)")
    axes.legend()


interactive(children=(FloatSlider(value=0.7, description='fracL', max=2.0999999999999996, min=-0.7), FloatSlid…